Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from statsmodels.formula.api import ols
import statsmodels.api as sm

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
%matplotlib inline

from custom_functions import remove_extremes

Import dataset that was cleaned and prepared in Data Preparation notebook

In [2]:
conn = sqlite3.connect('../../data/processed/main.db')
query = '''SELECT * FROM df_main'''
df_main = pd.read_sql(query, conn, index_col='index').reset_index(drop=True)
#df_main.drop('index', axis=1, inplace=True)
df_main.head(3)
y = 'SalePrice'

In [3]:
def remove_df_extremes(df, devct, drop_zeros=False):
    
    for col in df.columns:
        df[col] = [float(num) for num in df[col]]
        med = df[col].median()
        std = df[col].std()

        max_ = med + devct*std
        min_ = 0.0001 if drop_zeros else med - devct*std 

        df[col] = [x if ((x>min_) & (x<max_)) else np.nan for x in df[col]]
    df.dropna(inplace=True)
    return df

In [4]:
def produce_model(df, x, y, devct, drop_zeros=False):
    model_data = pd.concat([df[y], df[x]], axis=1)
    model_data_trimmed = remove_df_extremes(model_data, devct, drop_zeros)

    formula = y + ' ~ ' + '+'.join(xs)
    model = ols(formula, model_data_trimmed).fit()
    print(formula)
    
    return model, model_data_trimmed

In [118]:
df_main.columns

Index(['SalePrice', 'Parcel_ID', 'NbrLivingUnits', 'Stories', 'BldgGrade',
       'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor', 'SqFtUpperFloor',
       'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving', 'SqFtTotBasement',
       'SqFtFinBasement', 'FinBasementGrade', 'SqFtGarageBasement',
       'SqFtGarageAttached', 'DaylightBasement', 'SqFtOpenPorch',
       'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem', 'HeatSource',
       'BrickStone', 'ViewUtilization', 'Bedrooms', 'BathHalfCount',
       'Bath3qtrCount', 'BathFullCount', 'FpSingleStory', 'FpMultiStory',
       'FpFreestanding', 'FpAdditional', 'YrBuilt', 'YrRenovated', 'Condition',
       'SaleInstrument', 'SaleWarning'],
      dtype='object')

In [93]:
def qq(df, col):
    plt.figure();
    sm.graphics.qqplot(df[col], line='45', fit=True)
    plt.title('QQ plot of {col} values (Normality Assumption)');

In [94]:
def dist(df, x):
    plt.figure();
    sns.distplot(df[x])
    plt.title(f'Distribution of {x} Values')

In [167]:
from sklearn.linear_model import LinearRegression

def scatter(df, x, model):
    
    #x_new = pd.DataFrame([df[x].min(), df[x].max()], columns=[x])
    #prediction = model.predict(x_new)
#    print(x_new)
    X = df[x]
    Y = df[y]
    
    #print(Y.shape)
    y_pred = LinearRegression().fit(X,Y)#.predict(X)
    #plt.plot(X, y_pred)
    
    plt.figure();
    #plt.plot(x_new, prediction, color='r')
    sns.scatterplot(df[x], df[y], alpha=0.7)
    
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(f'Scatter plot: {x} vs. {y} (Linearity Assumption)');

xs = ['SqFtTotLiving', 'Bedrooms', 'YrBuilt', 'Condition', 'Stories']
model, df_trimmed = produce_model(df_main, xs, y, 3)
scatter(df_trimmed, xs[1], model)
#df_trimmed[xs[1]].values
#pd.DataFrame([df_trimmed[x].min(), df_trimmed[x].max()], columns=[x])

SalePrice ~ SqFtTotLiving+Bedrooms+YrBuilt+Condition+Stories


ValueError: Expected 2D array, got 1D array instead:
array=[3. 2. 3. ... 3. 4. 3.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [120]:
xs = ['SqFtTotLiving', 'Bedrooms', 'YrBuilt', 'Condition', 'Stories']


model, df_trimmed = produce_model(df_main, xs, y, 3)
model.summary()

# fig = plt.figure(figsize=(15,8))
# fig = sm.graphics.plot_regress_exog(model, 'YrBuilt', fig=fig)
# plt.show()



# x = 'YrBuilt'
# qq(df_trimmed, x);
# dist(df_trimmed, x)
# scatter(df_trimmed, x, model)

SalePrice ~ SqFtTotLiving+Bedrooms+YrBuilt+Condition+Stories


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     2602.
Date:                Thu, 04 Mar 2021   Prob (F-statistic):               0.00
Time:                        18:39:18   Log-Likelihood:            -2.4289e+05
No. Observations:               17405   AIC:                         4.858e+05
Df Residuals:                   17399   BIC:                         4.858e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      5.609e+06    1.7e+05     32.928      0.000    5.28e+06    5.94e+06
SqFtTotLiving   323.6675      3.575     90.536      0.000     316.660     330.675
Bedrooms      -6.204e+04   3266.486    -18.993      0.000   -6.84e+04   -5.56e+04
YrBuilt       -2806.6659     86.867    -32.310      0.000   -2976.933   -2636.399
Condition      7314.2520   3305.893      2.212      0.027     834.371    1.38e+04
Stories        1.209e+05   4938.021     24.486      0.000    1.11e+05    1.31e+05
==============================================================================
Omnibus:                     3159.841   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8400.888
Skew:                           0.987   Prob(JB):                         0.00
Kurtosis:                       5.773   Cond. No.                     2.35e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.35e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""